In [ ]:
#============================================================================
# BM25 예제임
# => 
#============================================================================
# bm25 설치
# !pip install rank_bm25
import konlpy
from konlpy.tag import Mecab
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm

contexts = [
    "세계 배달 피자 리더 도미노피자가 우리 고구마를 활용한 신메뉴를 출시한다.도미노피자는 오는 2월 1일 국내산 고구마와 4가지 치즈가 어우러진 신메뉴 `우리 고구마 피자`를 출시하고 전 매장에서 판매를 시작한다. 이번에 도미노피자가 내놓은 신메뉴 `우리 고구마 피자`는 까다롭게 엄선한 국내산 고구마를 무스와 큐브 형태로 듬뿍 올리고, 모차렐라, 카망베르, 체더 치즈와 리코타 치즈 소스 등 4가지 치즈와 와규 크럼블을 더한 프리미엄 고구마 피자다.",
    "피자의 발상지이자 원조라고 할 수 있는 남부의 나폴리식 피자(Pizza Napolitana)는 재료 본연의 맛에 집중하여 뛰어난 식감을 자랑한다. 대표적인 나폴리 피자로는 피자 마리나라(Pizza Marinara)와 피자 마르게리타(Pizza Margherita)가 있다.",
    "도미노피자가 삼일절을 맞아 '방문포장 1+1' 이벤트를 진행한다. 이번 이벤트는 도미노피자 102개 매장에서 3월 1일 단 하루 동안 방문포장 온라인, 오프라인 주문 시 피자 1판을 더 증정하는 이벤트다. 온라인 주문 시 장바구니에 2판을 담은 후 할인 적용이 가능하며, 동일 가격 또는 낮은 가격의 피자를 고객이 선택하면 무료로 증정한다."
]

def tokenizer(sent):
  return sent.split(" ")

# 입력된 contexts를 mecab을 이용하여 형태소 추출 후 " " 붙여서 형태소 문장을 만듬.
# Mecab 선언
mecab = Mecab()

mecab_contexts=[]
for context in tqdm(contexts):
    temp = mecab.morphs(context)   # ['세계', '배달', '피자', '리더', '도미노피자','가'..] 식으로 temp 리스트가 생성됨
    #print(temp)
    sentence = " ".join(temp)      # 위 temp 리스트를 공백을 넣어서 한문장으로 합침 ['세계 배달 피자 리더 도미노피자 가 ...]
    mecab_contexts.append(sentence)
    
print(mecab_contexts)
print(len(mecab_contexts))        


# tokeniaer 후 인덱싱
tokenized_corpus = [tokenizer(doc) for doc in mecab_contexts]
bm25 = BM25Okapi(tokenized_corpus)

print(bm25.doc_len)
print(bm25.doc_freqs)

In [ ]:
#idf : 토큰의 inverse term frequency를 계산해둠
# bm25.idf

In [ ]:
query = "고구마 피자 신메뉴"
tokenized_query = tokenizer(query)

# 쿼리에 따른 스코어 구함
doc_scores = bm25.get_scores(tokenized_query)
doc_scores

In [ ]:
# 쿼리 문장 구함
bm25.get_top_n(tokenized_query, mecab_contexts, n=2)